In [13]:
import os
import mlflow

# Устанавливаем локальное хранилище
mlflow.set_tracking_uri('file:./mlflow_experiments_store')

# Создаём эксперимент "Default" если он не существует
experiment = mlflow.get_experiment_by_name("Default")
if experiment is None:
    experiment_id = mlflow.create_experiment("Default")
else:
    experiment_id = experiment.experiment_id

# Логируем тестовую метрику и артефакт
with mlflow.start_run(run_name='Default', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    mlflow.log_metric("test_metric", 0)
    mlflow.log_artifact("test_artifact.txt", "test_artifact")

print(f"Run id запуска: {run_id}")

print(f"Experiment ID: {experiment_id}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

Run id запуска: 7c25a5b85674482ca81f592a5def92fa
Experiment ID: 509864494489441911
Tracking URI: file:./mlflow_experiments_store
